In [1]:
# Retrieve the video's auto-generated subtitles and save them to a file
!mkdir -p bin
!mkdir -p data
!if [ ! -f bin/youtube-dl ]; then curl -L https://github.com/yt-dlp/yt-dlp/releases/download/2023.06.22/yt-dlp -o bin/youtube-dl; fi
!chmod a+rx bin/youtube-dl

video_id = "mXH7FUIfoMs"
!youtube-dl --write-auto-sub --sub-lang en --skip-download -o "data/{video_id}" https://www.youtube.com/watch?v={video_id}


[youtube] Extracting URL: https://www.youtube.com/watch?v=mXH7FUIfoMs
[youtube] mXH7FUIfoMs: Downloading webpage
[youtube] mXH7FUIfoMs: Downloading ios player API JSON
[youtube] mXH7FUIfoMs: Downloading android player API JSON
[youtube] mXH7FUIfoMs: Downloading m3u8 information
[info] mXH7FUIfoMs: Downloading subtitles: en
[info] mXH7FUIfoMs: Downloading 1 format(s): 313+251
Deleting existing file data/mXH7FUIfoMs.en.vtt
[info] Writing video subtitles to: data/mXH7FUIfoMs.en.vtt
[download] Destination: data/mXH7FUIfoMs.en.vtt
[download] 100% of  295.53KiB in 00:00:00 at 1.69MiB/s)


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/
  Using cached openai-0.26.4-py3-none-any.whl
  Attempting uninstall: openai
    Found existing installation: openai 0.27.8
    Uninstalling openai-0.27.8:
      Successfully uninstalled openai-0.27.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
semantic-kernel 0.3.1.dev0 requires openai<0.28.0,>=0.27.0, but you have openai 0.26.4 which is incompatible.


In [3]:
from collections import defaultdict
from datetime import timedelta
import webvtt


video_id = "mXH7FUIfoMs"
file = 'data/' + video_id+'.en.vtt'

captions = webvtt.read(file)


result = defaultdict(str)

last_words = []

for caption in captions:
    start_time = caption.start
    hh, mm, ss_ms = start_time.split(':')
    ss, ms = ss_ms.split('.')

    key = int(hh) * 60 + int(mm)

    current_words = caption.text.strip().split()

    overlap_index = 0
    for i in range(1, min(len(last_words), len(current_words)) + 1):
        if last_words[-i:] == current_words[:i]:
            overlap_index = i

    result[key] += " " + " ".join(current_words[overlap_index:]).strip()

    last_words = current_words

transscript = ""

for key in sorted(result.keys()):
    minute = f"{key:02d}:00:00"
    data = f"{minute} {result[key].strip()}" 
    transscript += data + "\n"


print (transscript)

00:00:00 hey everyone in this video I want to  talk about Azure container storage  because we always talk about containers  we talk about pods and we think of them  as chickens we have this pod and this  pod Springs into life  it does some specific function then it  disappears so they may be very  short-lived but that's not always the  case sometimes we have containers that  have to have some durable state  maybe it's running a database but it has  to be able to have this storage that  persist beyond the life cycle of the  container and so we absolutely have  these scenarios where hey I do need some  storage  that it can use  that is durable  and we have mechanisms to do that today  so the way this would work right now is  well we have this concept of persistent  volumes
01:00:00 and that persistent volume is backed by  some durable storage in the Azure world  if it was just single access it could be  an Azure disk if it was multi-pod access  I can use things like Azure files Azure  Ne

In [4]:
from azure.identity import AzureDeveloperCliCredential
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *
from azure.search.documents import SearchClient
from azure.ai.formrecognizer import DocumentAnalysisClient
import openai
import os



# READ ACS_KEY from ENV
acs_key = os.environ.get('ACS_KEY')
acs_instance = os.environ.get('ACS_INSTANCE')
openai_key = os.environ.get('AZURE_OPENAI_KEY')
openai_deployment = os.environ.get('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')
openai_instance = os.environ.get('AZURE_OPENAI_INSTANCE')

openai.api_type = "azure"
openai.api_key = openai_key 
openai.api_base = f"https://{openai_instance}.openai.azure.com"
openai.api_version = "2022-12-01"

# search_cred should be TokenCredential
auth_credentials = AzureKeyCredential(acs_key)

def initialize_search_index():
    search_client = SearchIndexClient(endpoint=f"https://{acs_instance}.search.windows.net/",
                                      credential=auth_credentials)
    if "embeddings" not in search_client.list_index_names():
        index_structure = SearchIndex(
            name="embeddings",
            fields=[
                SimpleField(name="vector_id", type="Edm.String", key=True),
                SearchableField(name="payload", type="Edm.String", analyzer_name="en.microsoft"),
                SearchableField(name="additional_metadata", type="Edm.String", analyzer_name="en.microsoft"),
                SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
                            hidden=False, searchable=True, filterable=False, sortable=False, facetable=False,
                            dimensions=1536, vector_search_configuration="default"), 
                SimpleField(name="timestamp", type="Edm.DateTimeOffset", filterable=True, facetable=True),
            ],
            semantic_settings=SemanticSettings(
                configurations=[SemanticConfiguration(
                    name='standard',
                    prioritized_fields=PrioritizedFields(
                        title_field=None, prioritized_content_fields=[SemanticField(field_name='payload')]))]),
                vector_search=VectorSearch(
                    algorithm_configurations=[
                        VectorSearchAlgorithmConfiguration(
                            name="default",
                            kind="hnsw",
                            hnsw_parameters=HnswParameters(metric="cosine") 
                        )
                    ]
                )        
            )
        print(f"Initializing search index")
        search_client.create_index(index_structure)
    else:
        print(f"Search index already exists")


initialize_search_index()


Search index already exists


In [5]:
import re


def create_embeddings(transscript):
    for line in transscript.splitlines():
        timecode = line[:8]
        text = line[9:]
        content = f"YouTube-ID: {video_id} \nTimecode: {timecode} \nText: {text}"
        videoid = video_id
        yield {
            "vector_id": re.sub("[^0-9a-zA-Z_-]","_",f"{video_id}-{timecode}"),
            "payload": content,
            "vector": openai.Embedding.create(engine=openai_deployment, input=text)["data"][0]["embedding"],
        }

def index(embedding_data, batch_size=1000):
    search_client = SearchClient(endpoint=f"https://{acs_instance}.search.windows.net/",
                                    index_name="embeddings",
                                    credential=auth_credentials)

    current_index = 0
    current_batch = []
    for embedding in embedding_data:
        current_batch.append(embedding)
        current_index += 1
        if current_index % batch_size == 0:
            report_status(search_client, current_batch)
            current_batch = []

    if len(current_batch) > 0:
        report_status(search_client, current_batch)

def report_status(search_client, batch):
    results = search_client.upload_documents(documents=batch)
    successful_uploads = sum(1 for result in results if result.succeeded)
    print(f"\tIndexed {len(results)}, {successful_uploads} succeeded")

embeddings = create_embeddings(transscript)
index(embeddings)

	Indexed 41, 41 succeeded
